# FABRIC Example: Map of Resources


## Configure the Environment

This process may take a moment, as the folium mapping package must be installed to the kernel.

In [ ]:
import os
os.system('conda install folium')
from fabrictestbed.slice_manager import SliceManager, Status
import json

credmgr_host = os.environ['FABRIC_CREDMGR_HOST']
orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']
print(f"CM Host: {credmgr_host} Orchestrator Host: {orchestrator_host}")

#Configure SSH Key
ssh_key = None
with open ("/home/fabric/.ssh/id_rsa.pub", "r") as myfile:
    ssh_key=myfile.read().strip()

### Save the Initial Refresh Token

In [ ]:
#Retrieve or set the refresh token (exprires 24 hours after login)
not_found=False
fabric_refresh_token=None
%store -r fabric_refresh_token

if fabric_refresh_token is None:
    fabric_refresh_token=os.environ['CILOGON_REFRESH_TOKEN']
    %store fabric_refresh_token
print("Fabric Refresh Token {}".format(fabric_refresh_token))
print("CILOGON_REFRESH_TOKEN environment variable: {}".format(os.environ['CILOGON_REFRESH_TOKEN']))

## Create and Initialize Slice Manager Object
Users can request tokens with different Project and Scopes by altering `project_name` and `scope` parameters in the refresh call below.

In [ ]:
slice_manager = SliceManager(oc_host = orchestrator_host, cm_host = credmgr_host, project_name = 'all', scope = 'all')
slice_manager.initialize()

### Refresh the ID Token

ID Tokens expire one hour after refresh. 

In [ ]:
try:
    id_token, refresh_token = slice_manager.refresh_tokens()
except Exception as e:
    print("Exception occurred while getting tokens:{}".format(e))

fabric_refresh_token=slice_manager.get_refresh_token()
print()
print("New Refresh Token: {}".format(fabric_refresh_token))
print()
print("Stored new Refresh Token")
%store fabric_refresh_token

### Query Slices

In [ ]:
status, slices = slice_manager.slices()

print("Response Status {}".format(status))
if status == Status.OK:
    print("Slices {}".format(slices))
else:
    print(f"Failure: {slices}")

### Get Slice Information, Query Slivers From Slices

In [ ]:
slice_ids = [slice.slice_id for slice in slices]
slice_names = [slice.slice_name for slice in slices]
slice_lease_ends = [slice.lease_end for slice in slices]
slice_graph_ids = [slice.graph_id for slice in slices]
clean_slice_lease_ends = []

months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
for sle in slice_lease_ends:
    year = sle[:4]
    month = months[int(sle[5:7]) - 1]
    day = sle[8:10]
    if day[0] == '0':
        day = day[1]
    hour = sle[11:13]
    ampm = "AM" if int(hour) <= 12 else "PM"
    if hour[0] == '0':
        hour = hour[1]
    if int(hour) > 12:
        hour = str(int(hour) - 12)
    minute = sle[14:16]
    second = sle[17:19]
    date = month + ' ' + day + ', ' + year + f' [{hour}:{minute}:{second} {ampm}]'
    clean_slice_lease_ends.append(date)
    

for slice_id in slice_ids:
    status, slivers = slice_manager.slivers(slice_id = slice_id)
    
    print(f'\nSlice Name: {slice_names[slice_ids.index(slice_id)]}\n')
    print("Response Status {}".format(status))
    if status == Status.OK:
        print("Slivers {}".format(slivers))
    else:
        print(f"Failure: {slivers}")

### Create and Save Map

- Click 'Simple' to view map of all resources or 'Advanced' to view breakdown
- Click check boxes to switch between slices
- Click markers to view details about individual resources

In [ ]:
status, advertised_topology = slice_manager.resources()
site_dict = {}

for t in advertised_topology.sites.values():
    name = t.name 
    loc_info = t.get_property("location")
    if loc_info is not None:
        loc = loc_info.to_latlon()
        site_dict[name] = loc
        
import folium
from folium.plugins import MarkerCluster
from folium.features import DivIcon
from folium.raster_layers import WmsTileLayer
import branca
from branca.element import Template, MacroElement
import ipywidgets as widgets
from IPython.display import clear_output
from math import sqrt

template = """
        {% macro html(this, kwargs) %}

        <!doctype html>
        <html lang="en">
        <head>
          <meta charset="utf-8">
          <meta name="viewport" content="width=device-width, initial-scale=1">
          <title>jQuery UI Draggable - Default functionality</title>
          <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

          <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
          <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>

          <script>
          $( function() {
            $( "#maplegend" ).draggable({
                            start: function (event, ui) {
                                $(this).css({
                                    right: "auto",
                                    top: "auto",
                                    bottom: "auto"
                                });
                            }
                        });
        });

          </script>
        </head>
        <body>


        <div id='maplegend' class='maplegend' 
            style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
             border-radius:6px; padding: 10px; font-size:15px;font-family: Arial; right: 20px; bottom: 20px;'>

        <div class='legend-title'>Legend</div>
        <div class='legend-scale'>
          <ul class='legend-labels'>
            <li><span style='background:green;opacity:0.8;'></span>Active</li>
            <li><span style='background:orange;opacity:0.8;'></span>Closing</li>
            <li><span style='background:red;opacity:0.8;'></span>Closed</li>

          </ul>
        </div>
        </div>

        </body>
        </html>

        <style type='text/css'>
          .maplegend .legend-title {
            text-align: left;
            margin-bottom: 5px;
            font-weight: bold;
            font-size: 90%;
            }
          .maplegend .legend-scale ul {
            margin: 0;
            margin-bottom: 5px;
            padding: 0;
            float: left;
            list-style: none;
            }
          .maplegend .legend-scale ul li {
            font-size: 80%;
            list-style: none;
            margin-left: 0;
            line-height: 18px;
            margin-bottom: 2px;
            }
          .maplegend ul.legend-labels li span {
            display: block;
            float: left;
            height: 16px;
            width: 30px;
            margin-right: 5px;
            margin-left: 0;
            border: 1px solid #999;
            }
          .maplegend .legend-source {
            font-size: 80%;
            color: #777;
            clear: both;
            }
          .maplegend a {
            color: #777;
            }

        </style>
        {% endmacro %}
        """

url = ('http://services.arcgisonline.com/arcgis/rest/services/World_Street_Map' + '/MapServer/tile/{z}/{y}/{x}')

def advanced(b):
    with output:
        clear_output()
        base = folium.Map(location = (38.12480976137421, -95.7129), zoom_start = 4.2, zoom_control = False, scrollWheelZoom = False, dragging = False, max_bounds = True)
        WmsTileLayer(url = url, layers = None, name = 'ESRI World Street Map', attr = 'ESRI World Street Map', control = False).add_to(base)

        for slice_id in slice_ids:
            status, slivers = slice_manager.slivers(slice_id = slice_id)
            status, t = slice_manager.get_slice(slice_id = slice_id)

            labels = {}
            for sliver in slivers:
                status, sliver_status = slice_manager.sliver_status(slice_id = slice_id, sliver_id = sliver.reservation_id)
                if sliver_status.resource_type == "VM":
                    name = sliver_status.name
                    site = sliver_status.site
                    capacities = sliver_status.capacities
                    components = [t.nodes[name].get_property('image_ref'), t.nodes[name].get_property('image_type')]
                    state = sliver_status.reservation_state
                    info = (site, capacities, components, state)
                    labels[name] = info

            locations = []
            popups = []
            icons = []

            for label in labels:
                locations.append(site_dict[labels[label][0]])
                popups.append(folium.Popup(branca.element.IFrame(html = f'<strong style="font-family: Arial">Sliver: {label} [{labels[label][3]}]</strong><div style="font-family: Arial; font-size: 10pt"><br>Site: {labels[label][0]}<br>Capacities: {labels[label][1]}<br>Image Ref: {labels[label][2][0]}<br>Image Type: {labels[label][2][1]}', width = 350, height = 130), max_width = 360))
                color = None
                if labels[label][3] == 'Active':
                    color = 'green'
                elif labels[label][3] == 'Closing':
                    color = 'orange'
                else:
                    color = 'red'
                icons.append(folium.Icon(color = color, icon = 'server', prefix = 'fa'))

            cluster = MarkerCluster(name = slice_names[slice_ids.index(slice_id)], 
                                    locations = locations, 
                                    popups = popups, 
                                    icons = icons,
                                    show = False)
            folium.map.Marker((54, -110), 
                              icon = DivIcon(icon_size = (300, 80),
                                             icon_anchor = (0, 0),
                                             html = f'<div style="font-size: 15pt;font-family: Arial;text-align: center;border: 1pt solid lightgray;background: ghostwhite">{slice_names[slice_ids.index(slice_id)]}<h3 style="font-size: 10pt;font-family: Arial">Lease End: {clean_slice_lease_ends[slice_ids.index(slice_id)]}')).add_to(cluster)
            cluster.add_to(base)
            
        folium.LayerControl(collapsed = True).add_to(base)  
        macro = MacroElement()
        macro._template = Template(template)
        base.get_root().add_child(macro)
        base.save('advanced_view.html')
        display(base)

def simple(b):
    with output:
        clear_output()
        base = folium.Map(location = (38.12480976137421, -95.7129), zoom_start = 4.2, zoom_control = False, scrollWheelZoom = False, dragging = False, max_bounds = True)
        WmsTileLayer(url = url, layers = None, name = 'ESRI World Street Map', attr = 'ESRI World Street Map', control = False).add_to(base)
        
        labels = {}
        for slice_id in slice_ids:
            status, slivers = slice_manager.slivers(slice_id = slice_id)
            status, t = slice_manager.get_slice(slice_id = slice_id)

            for sliver in slivers:
                status, sliver_status = slice_manager.sliver_status(slice_id = slice_id, sliver_id = sliver.reservation_id)
                if sliver_status.resource_type == "VM":
                    title = sliver_status.name + sliver_status.lease_end
                    name = sliver_status.name
                    site = sliver_status.site
                    capacities = sliver_status.capacities
                    components = [t.nodes[name].get_property('image_ref'), t.nodes[name].get_property('image_type')]
                    state = sliver_status.reservation_state
                    end = clean_slice_lease_ends[slice_ids.index(slice_id)]
                    parent = slice_names[slice_ids.index(slice_id)]
                    info = (site, capacities, components, state, end, name, parent)
                    labels[title] = info

        locations = []
        popups = []
        icons = []

        for label in labels:
            locations.append(site_dict[labels[label][0]])
            popups.append(folium.Popup(branca.element.IFrame(html = f'<strong style="font-family: Arial">Sliver: {labels[label][-2]} [{labels[label][3]}]</strong><br><div style="font-family: Arial; font-size: 10pt">Parent Slice: {labels[label][-1]}<br>Lease End: {labels[label][-3]}<br><br>Site: {labels[label][0]}<br>Capacities: {labels[label][1]}<br>Image Ref: {labels[label][2][0]}<br>Image Type: {labels[label][2][1]}', width = 350, height = 130), max_width = 360))
            color = None
            if labels[label][3] == 'Active':
                color = 'green'
            elif labels[label][3] == 'Closing':
                color = 'orange'
            else:
                color = 'red'
            icons.append(folium.Icon(color = color, icon = 'server', prefix = 'fa'))

        cluster = MarkerCluster(name = 'View All', 
                                locations = locations, 
                                popups = popups, 
                                icons = icons,
                                show = False)

        cluster.add_to(base)
        macro = MacroElement()
        macro._template = Template(template)
        base.get_root().add_child(macro)
        base.save('simple_view.html')
        display(base)
        
simple_button = widgets.Button(description = 'Simple', disabled = False, tooltip = 'Click for simple view')
advanced_button = widgets.Button(description = 'Advanced', disabled = False, tooltip = 'Click for advanced view')
simple_button.on_click(simple)
advanced_button.on_click(advanced)
output = widgets.Output()

buttons = widgets.HBox([simple_button, advanced_button])
widgets.VBox([buttons, output])